In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pli
import seaborn as sns
import os
import tensorflow as tf
from PIL import Image
import torch
from torch import nn
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
from sklearn.metrics import classification_report

In [ ]:
def clean_colab_memory(path):
    !rm -r {path}

def save_to_drive(src, dest):
    !cp -r {src} {dest}

# Esempio di utilizzo
# clean_colab_memory('/content/nih_chest_xray_subset')
# save_to_drive('/content/nih_chest_xray_subset', '/content/drive/My Drive/')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

labels_path = "/content/drive/MyDrive/nih_chest_xray_subset/sample_labels.csv"
labels_df = pd.read_csv(labels_path)

In [ ]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  5606 non-null   object 
 1   Finding Labels               5606 non-null   object 
 2   Follow-up #                  5606 non-null   int64  
 3   Patient ID                   5606 non-null   int64  
 4   Patient Age                  5606 non-null   object 
 5   Patient Gender               5606 non-null   object 
 6   View Position                5606 non-null   object 
 7   OriginalImageWidth           5606 non-null   int64  
 8   OriginalImageHeight          5606 non-null   int64  
 9   OriginalImagePixelSpacing_x  5606 non-null   float64
 10  OriginalImagePixelSpacing_y  5606 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 481.9+ KB


In [ ]:
labels_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [ ]:
labels_df.shape

(5606, 11)

In [ ]:
labels_df['Finding Labels'].value_counts()

,count
Finding Labels,
No Finding,3044
Infiltration,503
Effusion,203
Atelectasis,192
Nodule,144
...,...
Atelectasis|Edema|Effusion|Infiltration|Pneumonia,1
Atelectasis|Consolidation|Edema|Infiltration|Pneumonia,1
Atelectasis|Effusion|Hernia,1


In [ ]:
data = []

for _, row in labels_df.iterrows():
    image_path = os.path.join('/content/drive/MyDrive/nih_chest_xray_subset/images', row['Image Index'])
    labels = row['Finding Labels'].split('|')
    data.append({'image_path': image_path, 'labels': labels})

print(data[:5])

[{'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_005.png', 'labels': ['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000013_026.png', 'labels': ['Cardiomegaly', 'Emphysema']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000017_001.png', 'labels': ['No Finding']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000030_001.png', 'labels': ['Atelectasis']}, {'image_path': '/content/drive/MyDrive/nih_chest_xray_subset/images/00000032_001.png', 'labels': ['Cardiomegaly', 'Edema', 'Effusion']}]


In [ ]:
num_class_1 = sum(["Infiltration" in r["labels"] for r in data])
print(num_class_1)

967


In [ ]:
num_class_0 = sum(["Infiltration" not in r["labels"] for r in data])
print(num_class_0)

4639


In [ ]:
def preprocess_item(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, label

paths = [item['image_path'] for item in data]
labels = [1 if "Infiltration" in item["labels"] else 0 for item in data]

# Preprocessing con parallelismo
dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
dataset = dataset.map(lambda x, y: preprocess_item(x, y), num_parallel_calls=tf.data.AUTOTUNE)

dataset_nobatch = dataset
dataset = dataset.cache()

In [ ]:
# Shuffle per migliorare il training
dataset = dataset.shuffle(buffer_size=1000)  # Cambia il buffer in base al dataset

# Creazione dei batch # Prefetch per caricare il batch successivo in parallelo
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
for images, labels in dataset.take(1):
    print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")

Batch shape: (64, 224, 224, 3), Labels shape: (64,)


In [ ]:
# loaded_dataset = tf.data.Dataset.load("/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

In [ ]:
#total_images = 0
#for images, labels in dataset:
#   total_images += images.shape[0]
#
#print(f"Total_number_of: {total_images}")

In [ ]:
# tf.data.experimental.save(dataset,"/content/drive/MyDrive/nih_chest_xray_subset/processed_dataset")

# TensorFlow Keras

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False # don't update base_model weights during training

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),  # Layer intermedio
    tf.keras.layers.Dense(1, activation='sigmoid') # Layer finale
])

# GlobalMaxPooling2D: Perde un po' di accuratezza complessiva, ma migliora nel riconoscimento della classe 1 rispetto a GlobalAveragePooling2D
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
dataset_size = len(dataset)

test_size = int(dataset_size * 0.2)
validation_size = int(dataset_size * 0.1)
train_size = dataset_size - test_size - validation_size

test_set = dataset.take(test_size)  # Primi 20% per il test
validation_set = dataset.skip(test_size).take(validation_size)  # Successivi 20% per validazione
train_set = dataset.skip(test_size + validation_size)  # Resto per il training

In [ ]:
print(f"Dataset totale (batch): {dataset_size}")
print(f"Test set (batch): {test_size}")
print(f"Validation set (batch): {validation_size}")
print(f"Training set (batch): {train_size}")

Dataset totale (batch): 88
Test set (batch): 17
Validation set (batch): 8
Training set (batch): 63


In [ ]:
history = model.fit(train_set, epochs=4, validation_data = validation_set)

Epoch 1/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 314s 3s/step - accuracy: 0.7468 - loss: 1.1884 - val_accuracy: 0.8867 - val_loss: 0.3631
Epoch 2/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - accuracy: 0.7985 - loss: 0.5537 - val_accuracy: 0.8535 - val_loss: 0.4095
Epoch 3/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.8184 - loss: 0.4813 - val_accuracy: 0.8066 - val_loss: 0.4669
Epoch 4/4
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.8278 - loss: 0.4477 - val_accuracy: 0.8672 - val_loss: 0.3802


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.8469 - loss: 0.4347
Test Loss: 0.41805461049079895, Test Accuracy: 0.8547794222831726


In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype(int).flatten())

print(classification_report(y_true, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       950
           1       0.50      0.01      0.03       138

    accuracy                           0.87      1088
   macro avg       0.69      0.51      0.48      1088
weighted avg       0.83      0.87      0.82      1088



This is due to the imbalance of the dataset:  
the model almost always predicts 0, largely ignoring class 1.  
On class 1 we experienced:  
Precision = 50%: The model makes few predictions like 1, but half of these predictions are correct.  
Recall = 2%: The model identifies only 2% of the real class 1 examples

# Pytorch

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CovNet(nn.Module):
  def __init__(self, channels, classes):
    super(CovNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=channels, out_channels=32, kernel_size=(5,5), padding='same')
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5), padding='same')
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(in_features=64*56*56, out_features=128)
    self.relu3 = nn.ReLU()

    self.fc2 = nn.Linear(in_features=128, out_features=classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = self.maxpool1(x)
    x = nn.functional.relu(self.conv2(x))
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = nn.functional.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.softmax(x)
    return x

In [ ]:
learning_rate = 0.001
num_epochs = 10
num_classes = 2
batch_size = 64
criterion = nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model = CovNet(channels=3, classes=2).to(device)

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ChestXRayDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        """
        Args:
            image_paths (list): List of file paths to images.
            labels (list): List of labels corresponding to the images.
            transform (callable, optional): Transform to be applied on an image.
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)

        # Get the label
        label = self.labels[idx]

        return image, label

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Example CSV file containing image paths and labels
csv_file = '/content/drive/MyDrive/nih_chest_xray_subset/sample_labels.csv'  # Replace with your file path
labels_df = pd.read_csv(csv_file)

# Extract image paths and labels
data = []
labels = []  # Separate list for labels

for _, row in labels_df.iterrows():
    image_path = os.path.join('/content/drive/MyDrive/nih_chest_xray_subset/images', row['Image Index'])
    label = 1 if "Infiltration" in row['Finding Labels'].split('|') else 0  # Binary label
    data.append(image_path)  # Append only the image path
    labels.append(label)  # Append the binary label

# Split into train, test, and validation sets
train_size = int(0.7 * len(data))
test_size = int(0.2 * len(data))
val_size = len(data) - train_size - test_size

train_paths, train_labels = data[:train_size], labels[:train_size]
test_paths, test_labels = data[train_size:train_size+test_size], labels[train_size:train_size+test_size]
val_paths, val_labels = data[train_size+test_size:], labels[train_size+test_size:]


# Create datasets
train_dataset = ChestXRayDataset(train_paths, train_labels, transform=transform)
test_dataset = ChestXRayDataset(test_paths, test_labels, transform=transform)
val_dataset = ChestXRayDataset(val_paths, val_labels, transform=transform)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import time
import copy

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    import time
    import copy
    import torch

    since = time.time()

    test_acc_history = []
    test_loss_history = []
    train_acc_history = []
    train_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Dynamically determine dataset size
            dataset_size = sum(1 for _ in dataloaders[phase])

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # Backward pass and optimization in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'test':
                test_acc_history.append(epoch_acc.cpu().numpy())
                test_loss_history.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc.cpu().numpy())
                train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    history_dict = {'train_loss': train_loss_history, 'train_accuracy': train_acc_history,
                    'test_loss': test_loss_history, 'test_accuracy': test_acc_history}
    return model, history_dict


In [ ]:
train_model(model,
            {'train':train_loader},
            criterion=criterion,
            optimizer=optimizer,
            num_epochs=5)

Epoch 1/5


KeyboardInterrupt: 

# PYTORCH mobilenet_v2

In [ ]:
class ChestXRayDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data[idx]['image_path']
        label = 1 if "Infiltration" in self.data[idx]['labels'] else 0
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# Trasformazioni
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

# Dataset e DataLoader
dataset_pytorch = ChestXRayDataset(data, transform=transform)
train_size = int(0.7 * len(dataset_pytorch))
test_size = len(dataset_pytorch) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset_pytorch, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Modello pre-addestrato
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 1)  # Output binario
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Loss e optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/4, Loss: 0.4765
Epoch 2/4, Loss: 0.4357
Epoch 3/4, Loss: 0.4222
Epoch 4/4, Loss: 0.4099


In [ ]:
# Validation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = (torch.sigmoid(outputs) > 0.5).int()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Test Classification Report:")
print(classification_report(y_true, y_pred))

Test Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.98      0.90      1402
           1       0.04      0.00      0.01       280

    accuracy                           0.82      1682
   macro avg       0.44      0.49      0.45      1682
weighted avg       0.70      0.82      0.75      1682



# DATA AUGMENTATION TESTS

# TensorFlow Keras (with DATA AUGMENTATION)

In [ ]:
# Filtra solo immagini di classe 1
class_1_data = [item for item in data if "Infiltration" in item["labels"]]
class_1_paths = [item['image_path'] for item in class_1_data]
class_1_labels = [1] * len(class_1_data)

class_1_dataset = tf.data.Dataset.from_tensor_slices((class_1_paths, class_1_labels))

# Funzione di augmentation
def augment_image(image):
    # Operazioni di augmentazione
    image = tf.image.random_flip_left_right(image)  # Flip orizzontale
    image = tf.image.random_flip_up_down(image)    # Flip verticale
    image = tf.image.rot90(image, k=np.random.randint(1, 4))  # Rotazione casuale
    return image

# Funzione di preprocessamento e augmentation
def preprocess_and_augment(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    image = augment_image(image)  # Applica augmentation
    return image, label

# Dataset augmentato per la classe 1 (ogni immagine originale produce una immagine augmentata)
augmented_class_1_dataset = class_1_dataset.map(preprocess_and_augment, num_parallel_calls=tf.data.AUTOTUNE)
print(len(augmented_class_1_dataset))

967


In [ ]:
print(len(augmented_class_1_dataset))
print(len(dataset_nobatch))
# Combina immagini originali e augmentate della classe 1
combined_class_1_dataset = dataset_nobatch.concatenate(augmented_class_1_dataset)
print(len(combined_class_1_dataset))

967
5606
6573


In [ ]:
class_counts = {"class_0": 0, "class_1": 0}
for _, label in combined_class_1_dataset:
    if label.numpy() == 0:
        class_counts["class_0"] += 1
    else:
        class_counts["class_1"] += 1

print(f"Distribuzione delle classi: {class_counts}")

Distribuzione delle classi: {'class_0': 4639, 'class_1': 1934}


In [ ]:
# Applica il batching al dataset combinato
combined_class_1_dataset = combined_class_1_dataset.shuffle(buffer_size=1000)
combined_class_1_dataset = combined_class_1_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
dataset_size = len(combined_class_1_dataset)
test_size = int(dataset_size * 0.3)
train_size = dataset_size - test_size

test_set = combined_class_1_dataset.take(test_size)  # Primi 20% per il test
train_set = combined_class_1_dataset.skip(test_size)  # Resto per il training

In [ ]:
print(f"Dataset totale: {dataset_size}")
print(f"Test set: {test_size}")
print(f"Training set: {train_size}")

Dataset totale: 103
Test set: 30
Training set: 73


In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_set, epochs=4)

Epoch 1/4
73/73 ━━━━━━━━━━━━━━━━━━━━ 103s 735ms/step - accuracy: 0.7185 - loss: 0.9769
Epoch 2/4
73/73 ━━━━━━━━━━━━━━━━━━━━ 94s 732ms/step - accuracy: 0.7936 - loss: 0.5281
Epoch 3/4
73/73 ━━━━━━━━━━━━━━━━━━━━ 140s 727ms/step - accuracy: 0.7880 - loss: 0.5418
Epoch 4/4
73/73 ━━━━━━━━━━━━━━━━━━━━ 140s 681ms/step - accuracy: 0.8107 - loss: 0.4896


In [ ]:
test_loss, test_accuracy = model.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 43s 856ms/step - accuracy: 0.8069 - loss: 0.4452
Test Loss: 0.4664957821369171, Test Accuracy: 0.8005208373069763


In [ ]:
y_true = []  # Ground truth
y_pred = []  # Predizioni

# Itera su tutti i batch del test set
for images, labels in test_set:
    preds = model.predict(images)
    y_true.extend(labels.numpy())  # Aggiungi tutte le etichette vere
    y_pred.extend(preds.flatten())  # Aggiungi tutte le predizioni (probabilità)

# Converti y_true e y_pred in array NumPy
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Testa diverse soglie
thresholds = [0.3, 0.5, 0.7]
for threshold in thresholds:
    y_pred_class = (y_pred > threshold).astype(int)  # Applica la soglia
    print(f"Classification report for threshold = {threshold}")
    print(classification_report(y_true, y_pred_class))


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━

# Pytorch (with DATA AUGMENTATION)

In [ ]:
from torchvision import transforms
from torch.utils.data import ConcatDataset

# Trasformazioni con Data Augmentation
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Flip orizzontale
    transforms.RandomVerticalFlip(p=0.5),    # Flip verticale
    transforms.RandomRotation(degrees=45),   # Rotazione casuale entro 45°
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),  # Crop casuale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Trasformazioni base (senza augmentation)
base_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset originale con trasformazioni base
base_dataset = ChestXRayDataset(data, transform=base_transforms)

# Dataset augmentato solo per la classe 1
class_1_data = [item for item in data if "Infiltration" in item["labels"]]
augmented_dataset = ChestXRayDataset(class_1_data, transform=augmentation_transforms)

# Combina il dataset originale e quello augmentato
combined_dataset = ConcatDataset([base_dataset, augmented_dataset])

# DataLoader
train_size = int(0.8 * len(combined_dataset))
test_size = len(combined_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(combined_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Modello pre-addestrato
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 1)  # Output binario
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Loss e optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (come definito in precedenza)
num_epochs = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Valutazione sul validation e test set
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = (torch.sigmoid(outputs) > 0.5).int()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Test Classification Report:")
print(classification_report(y_true, y_pred))


Epoch 1/4, Loss: 0.4376
Epoch 2/4, Loss: 0.3895
Epoch 3/4, Loss: 0.3785
Epoch 4/4, Loss: 0.3612
Test Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.93      0.87       913
           1       0.77      0.55      0.64       402

    accuracy                           0.81      1315
   macro avg       0.80      0.74      0.76      1315
weighted avg       0.81      0.81      0.80      1315

